In [38]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [39]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.model_selection import bayesian_search_forecaster_multiseries

In [40]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922
2012-01-04,25.895533,24.567708,27.252276
2012-01-05,21.379238,18.191667,20.357737


In [41]:
# Split data into train-val-test
# ==============================================================================
end_train = '2012-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2012-07-15 00:00:00   (n=197)
Test dates  : 2012-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=900)


In [42]:
# Create and train ForecasterRecursiveMultiSeries
# ==============================================================================
forecaster = ForecasterRecursiveMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 window_features    = RollingFeatures(stats=['mean', 'mean'], window_sizes=[24, 48]),
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

forecaster.fit(series=data_train)
forecaster

============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: LGBMRegressor 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
Window features: ['roll_mean_24', 'roll_mean_48'] 
Window size: 48 
Series encoding: ordinal 
Series names (levels): item_1, item_2, item_3 
Exogenous included: False 
Exogenous names: None 
Transformer for series: None 
Transformer for exog: None 
Weight function included: False 
Series weights: None 
Differentiation order: None 
Training range: 
    'item_1': ['2012-01-01', '2012-07-15'], 'item_2': ['2012-01-01', '2012-07-15'],
    'item_3': ['2012-01-01', '2012-07-15'] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: 
    {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0,
    'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1,
    'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0,
    'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None,
    'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0,
    'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1} 
fit_kwargs: {} 
Creation date: 2025-01-15 00:17:31 
Last fit date: 2025-01-15 00:17:31 
Skforecast version: 0.15.0 
Python version: 3.12.4 
Forecaster id: None

In [43]:
{k: np.sum(v) for k, v in forecaster.in_sample_residuals_.items()}

{'item_1': -0.013414809009656636,
 'item_2': 2.825167102096384,
 'item_3': -2.811744892477858,
 '_unknown_level': 7.400608868834979e-06}

In [44]:
{k: len(v) for k, v in forecaster.in_sample_residuals_.items()}

{'item_1': 149, 'item_2': 149, 'item_3': 149, '_unknown_level': 447}

In [45]:
{k: np.sum(v) for k, v in forecaster.in_sample_residuals_.items()}


{'item_1': -0.013414809009656636,
 'item_2': 2.825167102096384,
 'item_3': -2.811744892477858,
 '_unknown_level': 7.400608868834979e-06}

In [46]:
{k: len(v) for k, v in forecaster.in_sample_residuals_.items()}

{'item_1': 149, 'item_2': 149, 'item_3': 149, '_unknown_level': 447}

In [47]:
forecaster.out_sample_residuals_by_bin_

In [50]:
forecaster.out_sample_residuals_

{'item_1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'item_2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.

In [49]:
forecaster.set_out_sample_residuals(
    y_true=data_train.to_dict(orient="series"),
    y_pred=data_train.to_dict(orient="series")
)

aquiiiiiiiiiiiii


In [10]:
# Predictions and prediction intervals
# ==============================================================================
steps = 24

# Predictions for item_1
predictions_item_1 = forecaster.predict(steps=steps, levels='item_1')
display(predictions_item_1.head(3))

,level,pred
2012-07-16,item_1,23.611030
2012-07-17,item_1,24.752059
2012-07-18,item_1,25.458480


In [11]:
# Interval predictions for item_1 and item_2
predictions_intervals = forecaster.predict_interval(
    steps  = steps,
    levels = ['item_1', 'item_2'],
    n_boot = 50
)
predictions_intervals

TypeError: ForecasterRecursiveMultiSeries._create_predict_inputs() got an unexpected keyword argument 'use_binned_residuals'

In [10]:
[len(v) for v in forecaster.in_sample_residuals_.values()]

[879, 879, 879, 1000]

In [ ]:
forecaster.in_sample_residuals_by_bin_

AttributeError: 'ForecasterRecursiveMultiSeries' object has no attribute 'in_sample_residuals_by_bin_'